# OpenAI Fine-tuning 완전 가이드### Cadwell Korea 의료기기 브로셔 번역 모델 학습이 노트북은 다음 작업을 순서대로 진행합니다:1. OpenAI API 설정2. JSONL 데이터 파일 업로드3. Fine-tuning 작업 시작4. 학습 상태 모니터링5. 학습된 모델 테스트6. Backend에서 사용하는 코드 예시---

## 1️⃣ OpenAI 설치 및 API 키 설정

In [ ]:
# OpenAI 라이브러리 설치!pip install openai --upgrade

In [ ]:
# API 키 입력# OpenAI 플랫폼에서 발급받은 API 키를 입력하세요# https://platform.openai.com/api-keysimport osfrom openai import OpenAI# 여기에 API 키를 입력하세요API_KEY = ""  # 예: "sk-proj-..."# 또는 Colab Secrets 사용 (권장)# from google.colab import userdata# API_KEY = userdata.get('OPENAI_API_KEY')if not API_KEY:    raise ValueError("❌ API_KEY를 입력하세요!")# OpenAI 클라이언트 초기화client = OpenAI(api_key=API_KEY)print("✅ OpenAI 클라이언트가 초기화되었습니다.")

## 2️⃣ JSONL 학습 데이터 업로드좌측 파일 탭에서 `training_data.jsonl` 파일을 업로드하세요.

In [ ]:
# 업로드된 파일 확인import osTRAINING_FILE = "training_data.jsonl"if not os.path.exists(TRAINING_FILE):    print("❌ training_data.jsonl 파일을 먼저 업로드하세요!")    print("   좌측 파일 아이콘 > 업로드 버튼 클릭")else:    # 파일 내용 미리보기    import json        with open(TRAINING_FILE, 'r', encoding='utf-8') as f:        lines = f.readlines()        print(f"✅ 파일을 찾았습니다!")    print(f"   총 학습 예제 수: {len(lines)}개")    print(f"\n첫 번째 예제:")        first_example = json.loads(lines[0])    for msg in first_example['messages']:        print(f"\n[{msg['role'].upper()}]")        print(msg['content'][:100] + ('...' if len(msg['content']) > 100 else ''))

## 3️⃣ OpenAI Files API에 업로드

In [ ]:
# OpenAI Files API에 학습 데이터 업로드print("📤 OpenAI Files API에 업로드 중...")with open(TRAINING_FILE, "rb") as f:    training_file = client.files.create(        file=f,        purpose="fine-tune"    )print(f"✅ 업로드 완료!")print(f"   파일 ID: {training_file.id}")print(f"   파일명: {training_file.filename}")print(f"   상태: {training_file.status}")print(f"   크기: {training_file.bytes} bytes")# 파일 ID 저장 (나중에 사용)FILE_ID = training_file.id

## 4️⃣ Fine-tuning 작업 시작기본 모델: `gpt-4o-mini-2024-07-18`  학습 시간: 데이터 크기에 따라 수 분 ~ 수십 분 소요

In [ ]:
# Fine-tuning 작업 생성print("🚀 Fine-tuning 작업을 시작합니다...")print("   기본 모델: gpt-4o-mini-2024-07-18")print("   목적: Cadwell Korea 의료기기 브로셔 번역\n")fine_tuning_job = client.fine_tuning.jobs.create(    training_file=FILE_ID,    model="gpt-4o-mini-2024-07-18",    hyperparameters={        "n_epochs": 3,  # 에포크 수 (기본값: 3-4, 데이터가 많으면 1-2로 줄여도 됨)    },    suffix="cadwell-medical-ko"  # 모델명에 추가될 접미사)print(f"✅ Fine-tuning 작업이 생성되었습니다!")print(f"   작업 ID: {fine_tuning_job.id}")print(f"   상태: {fine_tuning_job.status}")print(f"   생성 시간: {fine_tuning_job.created_at}")# 작업 ID 저장JOB_ID = fine_tuning_job.idprint(f"\n⏳ 학습이 진행 중입니다. 다음 셀에서 상태를 모니터링하세요.")

## 5️⃣ 학습 상태 모니터링

In [ ]:
# 현재 상태 확인 (한 번만 실행)job_status = client.fine_tuning.jobs.retrieve(JOB_ID)print(f"📊 Fine-tuning 작업 상태")print(f"   작업 ID: {job_status.id}")print(f"   상태: {job_status.status}")print(f"   기본 모델: {job_status.model}")print(f"   생성 시간: {job_status.created_at}")if job_status.fine_tuned_model:    print(f"\n✅ 학습 완료!")    print(f"   Fine-tuned 모델 ID: {job_status.fine_tuned_model}")else:    print(f"\n⏳ 학습 진행 중... 잠시 후 다시 실행하세요.")        if job_status.status == "failed":        print(f"\n❌ 학습 실패!")        if job_status.error:            print(f"   오류: {job_status.error}")

In [ ]:
# 실시간 모니터링 (계속 실행)import timefrom datetime import datetimeprint("🔄 실시간 모니터링 시작 (Ctrl+C로 중단)\n")try:    while True:        job = client.fine_tuning.jobs.retrieve(JOB_ID)                current_time = datetime.now().strftime("%H:%M:%S")        print(f"[{current_time}] 상태: {job.status}", end="")                if job.status == "succeeded":            print(f"\n\n✅ 학습 완료!")            print(f"   Fine-tuned 모델 ID: {job.fine_tuned_model}")            FINE_TUNED_MODEL = job.fine_tuned_model            break        elif job.status == "failed":            print(f"\n\n❌ 학습 실패!")            if job.error:                print(f"   오류: {job.error}")            break        else:            print(" (진행 중...)")                time.sleep(30)  # 30초마다 확인        except KeyboardInterrupt:    print("\n\n⏸️  모니터링이 중단되었습니다.")    print("   학습은 계속 진행됩니다. 위 셀로 상태를 확인하세요.")

## 6️⃣ 학습 이벤트 로그 확인

In [ ]:
# Fine-tuning 이벤트 로그 조회from datetime import datetimeevents = client.fine_tuning.jobs.list_events(fine_tuning_job_id=JOB_ID, limit=20)print("📜 최근 20개 이벤트 로그:\n")for event in events.data:    timestamp = datetime.fromtimestamp(event.created_at).strftime("%Y-%m-%d %H:%M:%S")    print(f"[{timestamp}] {event.message}")

## 7️⃣ 학습된 모델 ID 확인 및 저장

In [ ]:
# 최종 Fine-tuned 모델 ID 확인final_job = client.fine_tuning.jobs.retrieve(JOB_ID)if final_job.fine_tuned_model:    FINE_TUNED_MODEL = final_job.fine_tuned_model        print("✅ Fine-tuned 모델 정보")    print("=" * 60)    print(f"모델 ID: {FINE_TUNED_MODEL}")    print("=" * 60)    print("\n이 모델 ID를 복사하여 Backend 코드에 사용하세요!")    print("\n예시:")    print(f'OPENAI_MODEL = "{FINE_TUNED_MODEL}"')        # 파일로 저장    with open("fine_tuned_model_id.txt", "w") as f:        f.write(FINE_TUNED_MODEL)    print("\n💾 모델 ID가 'fine_tuned_model_id.txt'에 저장되었습니다.")    else:    print("❌ Fine-tuned 모델이 아직 준비되지 않았습니다.")    print(f"   현재 상태: {final_job.status}")

## 8️⃣ 테스트 번역 (학습된 모델 사용)

In [ ]:
# Fine-tuned 모델로 번역 테스트def test_translation(english_text: str, model_id: str) -> str:    """    Fine-tuned 모델을 사용하여 영어를 한국어로 번역    """    response = client.chat.completions.create(        model=model_id,        messages=[            {                "role": "system",                "content": "Cadwell Korea 의료기기 브로셔 전문 번역가"            },            {                "role": "user",                "content": english_text            }        ],        temperature=0.3,  # 일관성을 위해 낮은 temperature 사용    )        return response.choices[0].message.content# 테스트 문장들test_sentences = [    "Cadwell's EMG solutions are designed for comprehensive neuromuscular diagnostics.",    "Our devices provide accurate and reliable measurements for clinical assessments.",    "The system integrates seamlessly with existing hospital infrastructure.",    "Advanced signal processing ensures optimal data quality."]print("🧪 Fine-tuned 모델 테스트\n")print("=" * 60)if 'FINE_TUNED_MODEL' not in locals():    print("❌ FINE_TUNED_MODEL이 설정되지 않았습니다.")    print("   위 셀을 먼저 실행하여 모델 ID를 가져오세요.")else:    for i, sentence in enumerate(test_sentences, 1):        print(f"\n[테스트 {i}]")        print(f"원문: {sentence}")                translation = test_translation(sentence, FINE_TUNED_MODEL)        print(f"번역: {translation}")        print("-" * 60)

## 9️⃣ 기본 모델과 비교 (선택사항)

In [ ]:
# Fine-tuned 모델 vs 기본 모델 비교test_text = "The integrated workflow provides efficient patient data management and reporting."print("📊 모델 비교 테스트")print("=" * 60)print(f"\n원문:\n{test_text}\n")# 기본 모델 (gpt-4o-mini)print("[기본 모델: gpt-4o-mini]")base_translation = test_translation(test_text, "gpt-4o-mini")print(f"{base_translation}\n")# Fine-tuned 모델if 'FINE_TUNED_MODEL' in locals():    print(f"[Fine-tuned 모델: {FINE_TUNED_MODEL}]")    finetuned_translation = test_translation(test_text, FINE_TUNED_MODEL)    print(f"{finetuned_translation}\n")        print("-" * 60)    print("\n💡 Fine-tuned 모델은 학습 데이터의 스타일과 용어를 반영합니다.")else:    print("\n⚠️  Fine-tuned 모델이 준비되지 않았습니다.")

## 🔟 Backend에서 사용하는 코드 (Python FastAPI)

In [ ]:
# Backend 코드 예시 출력if 'FINE_TUNED_MODEL' in locals():    backend_code = f'''# backend/app/services/translate_service.pyfrom openai import OpenAIimport os# Fine-tuned 모델 ID (환경 변수로 관리 권장)OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")FINE_TUNED_MODEL = "{FINE_TUNED_MODEL}"client = OpenAI(api_key=OPENAI_API_KEY)def translate_with_finetuned_model(english_text: str) -> str:    """    Fine-tuned 모델을 사용하여 영어를 한국어로 번역        Args:        english_text: 번역할 영어 텍스트            Returns:        번역된 한국어 텍스트    """    try:        response = client.chat.completions.create(            model=FINE_TUNED_MODEL,            messages=[                {{                    "role": "system",                    "content": "Cadwell Korea 의료기기 브로셔 전문 번역가"                }},                {{                    "role": "user",                    "content": english_text                }}            ],            temperature=0.3,            max_tokens=2000,        )                return response.choices[0].message.content            except Exception as e:        print(f"번역 오류: {{e}}")        raise# 사용 예시if __name__ == "__main__":    test_text = "The EMG system provides comprehensive diagnostic capabilities."    result = translate_with_finetuned_model(test_text)    print(f"원문: {{test_text}}")    print(f"번역: {{result}}")'''        print("📝 Backend 통합 코드")    print("=" * 60)    print(backend_code)    print("=" * 60)        # 파일로 저장    with open("backend_integration_code.py", "w", encoding="utf-8") as f:        f.write(backend_code)        print("\n💾 'backend_integration_code.py' 파일로 저장되었습니다.")else:    print("⚠️  먼저 Fine-tuned 모델 ID를 가져오세요.")

## 1️⃣1️⃣ 환경 변수 설정 (.env 파일)

In [ ]:
# .env 파일 예시 생성if 'FINE_TUNED_MODEL' in locals():    env_content = f'''# OpenAI API 설정OPENAI_API_KEY=sk-proj-your-api-key-hereOPENAI_MODEL={FINE_TUNED_MODEL}# 번역 제공자 설정TRANSLATION_PROVIDER=openai# 기타 설정FRONTEND_ORIGIN=http://localhost:5173'''        print("📄 .env 파일 예시")    print("=" * 60)    print(env_content)    print("=" * 60)        with open("example.env", "w", encoding="utf-8") as f:        f.write(env_content)        print("\n💾 'example.env' 파일로 저장되었습니다.")    print("   이 파일을 프로젝트 루트에 '.env'로 복사하고 API 키를 입력하세요.")

## 1️⃣2️⃣ 모든 Fine-tuning 작업 목록 조회

In [ ]:
# 모든 Fine-tuning 작업 조회from datetime import datetimejobs = client.fine_tuning.jobs.list(limit=10)print("📋 최근 Fine-tuning 작업 목록 (최대 10개)\n")print("=" * 80)for job in jobs.data:    created = datetime.fromtimestamp(job.created_at).strftime("%Y-%m-%d %H:%M:%S")        print(f"작업 ID: {job.id}")    print(f"상태: {job.status}")    print(f"기본 모델: {job.model}")    print(f"생성 시간: {created}")        if job.fine_tuned_model:        print(f"Fine-tuned 모델: {job.fine_tuned_model}")        print("-" * 80)

## 1️⃣3️⃣ Fine-tuned 모델 삭제 (선택사항)

In [ ]:
# ⚠️ 주의: 이 셀은 Fine-tuned 모델을 삭제합니다!# 실행하기 전에 model_to_delete 변수를 설정하세요.# 삭제할 모델 ID 입력model_to_delete = ""  # 예: "ft:gpt-4o-mini-2024-07-18:..."if model_to_delete:    confirm = input(f"정말로 '{model_to_delete}' 모델을 삭제하시겠습니까? (yes/no): ")        if confirm.lower() == "yes":        try:            client.models.delete(model_to_delete)            print(f"✅ 모델 '{model_to_delete}'이(가) 삭제되었습니다.")        except Exception as e:            print(f"❌ 삭제 실패: {e}")    else:        print("취소되었습니다.")else:    print("⚠️  model_to_delete 변수에 삭제할 모델 ID를 입력하세요.")

## 📚 추가 리소스- [OpenAI Fine-tuning 공식 문서](https://platform.openai.com/docs/guides/fine-tuning)- [OpenAI API Reference](https://platform.openai.com/docs/api-reference/fine-tuning)- [Fine-tuning 가격 정보](https://openai.com/pricing)---## ✅ 완료 체크리스트- [ ] OpenAI API 키 설정- [ ] training_data.jsonl 파일 업로드- [ ] Files API에 업로드- [ ] Fine-tuning 작업 시작- [ ] 학습 완료 확인- [ ] Fine-tuned 모델 ID 저장- [ ] 테스트 번역 실행- [ ] Backend 코드에 모델 ID 적용- [ ] .env 파일에 API 키와 모델 ID 설정---**🎉 축하합니다! Fine-tuning이 완료되었습니다.**이제 학습된 모델을 프로덕션 환경에서 사용할 수 있습니다.